In [11]:
from __future__ import division

import numpy as np
import pandas
import scipy
from sklearn import svm

import matplotlib.pyplot as plt

import nltk
from nltk import word_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/philippe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/philippe/nltk_data...


[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
english = pandas.read_csv("train_cap2018.csv")

In [9]:
target = english['level1']
target.head()
print(target.shape)

# data
data = english.loc[:, "fulltext":"punct"]
print(data.shape)

# words_data : 
text = english.loc[:, "fulltext"]
words_data = text.map(lambda t: word_tokenize(t))


# print(data.loc[3]["fulltext"][0])
# sentence = data[:3]["fulltext"][0]
# sentence = data.as_matrix()[27309, 0]
# print(sentence)
# words = word_tokenize(sentence)


(27310,)
(27310, 6)


In [203]:
words_data[27304]

['sixteen',
 'headphones',
 '.',
 'also',
 'there',
 'are',
 'thirty',
 'pens',
 'and',
 'fifteen',
 'pencils',
 '.',
 'also',
 'there',
 'are',
 'twelve',
 'desks',
 ',',
 'fourteen',
 'chairs',
 'and',
 'one',
 'table',
 '.',
 'bye']

In [30]:

tagged = nltk.pos_tag(words_data[27304])

print words_data[27305]

print len(set(words_data[27305]))

print len(words_data[27305])
print (len(set(words_data[27305]))/len(words_data[27305]))
print (len(set(words_data[3000]))/len(words_data[3000]))
print (len(set(words_data[2]))/len(words_data[2]))


def lexical_diversity(text):
    return len(set(text)) / len(text)


ld = words_data.map(lambda t: lexical_diversity(t))
ld


['Hi', 'my', 'name', "'s", 'Jean-Marc', '.', 'I', "'m", 'a', 'management', 'controller', '.', 'I', 'work', 'in', 'an', 'office', '.', 'My', 'favorite', 'day', 'in', 'Sunday', '.', 'I', 'get', 'up', 'at', 'eight', "o'clock", 'in', 'the', 'morning', 'on', 'Sunday', ',', 'have', 'a', 'shower', ',', 'brush', 'my', 'teeth', 'and', 'go', 'running', '.', 'I', 'have', 'lunch', 'at', 'twelve', "o'clock", 'in', 'the', 'afternoon', '.', 'I', 'go', 'swimming', 'in', 'the', 'evening', ',', 'I', 'meet', 'friends', '.', 'We', 'eat', 'in', 'a', 'restaurant', 'then', 'go', 'to', 'the', 'movies', 'and', 'watch', 'a', 'movie', '.', 'I', 'go', 'to', 'bed', 'at', 'eleven', "o'clock", '.']
52
91
0.571428571429
0.623853211009
0.69387755102


0        0.579767
1        0.609137
2        0.693878
3        0.647959
4        0.570732
5        0.619289
6        0.673759
7        0.504902
8        0.597285
9        0.586592
10       0.756757
11       0.503597
12       0.619718
13       0.518072
14       0.623711
15       0.666667
16       0.592814
17       0.632353
18       0.559748
19       0.638743
20       0.615385
21       0.734043
22       0.656863
23       0.600877
24       0.632124
25       0.542105
26       0.586022
27       0.592814
28       0.607595
29       0.617347
           ...   
27280    0.326531
27281    0.950000
27282    0.608108
27283    0.627907
27284    0.701754
27285    0.937500
27286    0.717391
27287    0.916667
27288    0.754717
27289    0.784314
27290    0.742857
27291    0.820000
27292    0.840909
27293    0.730769
27294    0.740741
27295    0.818182
27296    0.562500
27297    0.441176
27298    0.784314
27299    0.784615
27300    0.714286
27301    0.781250
27302    0.710526
27303    0.616438
27304    0

In [121]:
fig = plt.figure()
y = english.groupby('level1').sentences.count()
plt.bar(y.keys(), y.values)
plt.show()

In [5]:
# clf = svm.SVC(gamma=0.001, C=100.)
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0) 
data_cleaned = data.T.dropna().T

clf.fit(data_cleaned, target)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

In [6]:
data_cleaned.head()

print(data_cleaned[:1])
clf.predict(data_cleaned[:1])

   sentences  words  letters.all  syllables  punct
0         16    233         1116        363     32


array(['B1'], dtype=object)

In [7]:
from sklearn.model_selection import cross_val_score
%time listerr = cross_val_score(clf, data_cleaned, target, cv=5)
print listerr

CPU times: user 1min 28s, sys: 48 ms, total: 1min 29s
Wall time: 1min 50s
[0.5610247  0.56232839 0.57817649 0.5521978  0.54542125]


In [19]:
from sklearn.preprocessing import StandardScaler
data_standard = StandardScaler().fit_transform(data_cleaned)
corr = np.corrcoef(data_standard.T)
corr.shape

(5, 5)

In [20]:
import seaborn as sea
sea.heatmap(corr)
corr

array([[1.        , 0.66837766, 0.64175596, 0.64342692, 0.73276522],
       [0.66837766, 1.        , 0.98341281, 0.98538917, 0.71997432],
       [0.64175596, 0.98341281, 1.        , 0.99595057, 0.70399435],
       [0.64342692, 0.98538917, 0.99595057, 1.        , 0.71004385],
       [0.73276522, 0.71997432, 0.70399435, 0.71004385, 1.        ]])